In [2]:
import pandas as pd

In [3]:
sales = pd.read_csv("../DataSet/start_vgsales.csv")
steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]
steam.columns = ["player_id", "game_title", "type", "value"]

C:\Users\david\AppData\Local\Temp\ipykernel_8100\560633259.py:2: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  steam = pd.read_csv("../DataSet/start_steam-200k.csv", header=None).iloc[:,:-1]


In [4]:
steam = steam[steam["type"] == "purchase"]

In [5]:
df = steam.merge(sales, how='inner',left_on="game_title", right_on="Name")
df.head()

,player_id,game_title,type,value,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,151603712,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
1,151603712,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09
2,151603712,Fallout 4,purchase,1.0,1632,Fallout 4,PC,2015.0,Role-Playing,Bethesda Softworks,0.50,0.63,0.00,0.10,1.23
3,87445402,Fallout 4,purchase,1.0,110,Fallout 4,PS4,2015.0,Role-Playing,Bethesda Softworks,2.47,3.15,0.24,1.10,6.96
4,87445402,Fallout 4,purchase,1.0,288,Fallout 4,XOne,2015.0,Role-Playing,Bethesda Softworks,2.45,1.26,0.01,0.37,4.09


In [6]:
df2 = df[["player_id", "Genre", "value"]]
df3 = df2.groupby(["player_id", "Genre"]).count()

In [7]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np

In [8]:
users = df3.reset_index().drop(columns="value")
users["player_id"] = np.asarray(users["player_id"]).astype(np.int32)
unique_user_ids = users["player_id"].unique()
users

,player_id,Genre
0,86540,Action
1,86540,Adventure
2,86540,Racing
3,86540,Shooter
4,86540,Strategy
...,...,...
8953,99723205,Shooter
8954,99723205,Strategy
8955,99766416,Shooter
8956,99812428,Strategy


In [9]:
genres = users["Genre"].unique()
#genres = pd.DataFrame(data=genres, columns=['Genre'])
unique_genres = genres
genres

array(['Action', 'Adventure', 'Racing', 'Shooter', 'Strategy',
       'Role-Playing', 'Simulation', 'Sports', 'Fighting', 'Misc',
       'Puzzle', 'Platform'], dtype=object)

In [10]:
tf.random.set_seed(42)
users_tensor = tf.data.Dataset.from_tensor_slices(dict(users))
users_shuffled = users_tensor.shuffle(seed=42, buffer_size=100_000, reshuffle_each_iteration=False)

users_train = users_shuffled.take(8_000)
users_test = users_shuffled.skip(8_000).take(8958)

In [11]:
genres_tensor = tf.data.Dataset.from_tensor_slices(genres)

In [12]:
for r in genres_tensor.take(5):
    print(r)

tf.Tensor(b'Action', shape=(), dtype=string)
tf.Tensor(b'Adventure', shape=(), dtype=string)
tf.Tensor(b'Racing', shape=(), dtype=string)
tf.Tensor(b'Shooter', shape=(), dtype=string)
tf.Tensor(b'Strategy', shape=(), dtype=string)


In [13]:
from typing import Dict, Text
class MovieLensModel(tfrs.Model):
     # We derive from a custom base class to help reduce boilerplate. Under the hood,
     # these are still plain Keras Models.

    def __init__(
            self,
            user_model: tf.keras.Model,
            genre_model: tf.keras.Model,
            task: tfrs.tasks.Retrieval):
        super().__init__()

          # Set up user and movie representations.
        self.user_model = user_model
        self.genre_model = genre_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["player_id"])
        genre_embeddings = self.genre_model(features["Genre"])

        return self.task(user_embeddings, genre_embeddings)

In [43]:
user_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(mask_token=None, vocabulary=unique_user_ids),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, 10)
])
genre_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(mask_token=None, vocabulary=unique_genres),
    tf.keras.layers.Embedding(len(unique_genres) + 1, 10)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = genres_tensor.batch(128).map(genre_model)
    ),
)

In [44]:
cached_train = users_train.shuffle(100_000).batch(8192).cache()
cached_test = users_test.batch(4096).cache()

In [45]:
model = MovieLensModel(user_model, genre_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate = 0.1))

# Train for 3 epochs.
model.fit(cached_train, epochs=3)


Epoch 1/3
1/1 [==============================] - 3s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0634 - factorized_top_k/top_5_categorical_accuracy: 0.3890 - factorized_top_k/top_10_categorical_accuracy: 0.8090 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 71897.5391 - regularization_loss: 0.0000e+00 - total_loss: 71897.5391
Epoch 2/3
1/1 [==============================] - 1s 722ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1166 - factorized_top_k/top_5_categorical_accuracy: 0.4819 - factorized_top_k/top_10_categorical_accuracy: 0.8185 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 71883.0469 - regularization_loss: 0.0000e+00 - total_loss: 71883.0469
Epoch 3/3
1/1 [==============================] - 1s 864ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3217 - factorized_top_k/top_5_categorical_accuracy: 0.7651 - 

In [33]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    genres_tensor.batch(100).map(lambda title: (title, model.genre_model(title))))

# Get some recommendations.
_, titles = index(np.array([97782908]))
print(f"Top 3 recommendations for user 97782908: {titles[0, :10]}")
print(f"Scores: {_[0, :10]}")

Top 3 recommendations for user 97782908: [b'Misc' b'Sports' b'Strategy' b'Role-Playing' b'Adventure' b'Platform'
 b'Shooter' b'Fighting' b'Action' b'Puzzle']
Scores: [ 4.8108348e-03  4.5780106e-03  3.3074105e-03  2.3549767e-03
  1.4150678e-03  8.6006959e-04  3.3108619e-04 -8.8091096e-05
 -8.5499755e-04 -1.3946217e-03]


In [16]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 1s 629ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0292 - factorized_top_k/top_5_categorical_accuracy: 0.2453 - factorized_top_k/top_10_categorical_accuracy: 0.6378 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 7932.0552 - regularization_loss: 0.0000e+00 - total_loss: 7932.0552


{'factorized_top_k/top_1_categorical_accuracy': 0.02922755666077137,
 'factorized_top_k/top_5_categorical_accuracy': 0.2453027069568634,
 'factorized_top_k/top_10_categorical_accuracy': 0.6377870440483093,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 7932.05517578125,
 'regularization_loss': 0,
 'total_loss': 7932.05517578125}